<a href="https://colab.research.google.com/github/Huy1902/HAC/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import os
from IPython.display import display
import csv

In [3]:
path_to_data = 'rl_agent/dataset/mdp/interaction.csv'
path_to_item = 'rl_agent/dataset/mdp/tracks_features.csv'
# path_to_users = 'ml-1m/users.dat'
path_to_dataset = 'mdp'
path_to_output = 'rl_agent/dataset/mdp/'
seed = 2025

## Extract train and test dataset

In [3]:
data = pd.read_csv(path_to_data, sep=',')
data

,user_id,item_id,exists,timestamp
0,mpd_pid_0,6I9VzXrHxO9rA9A5euc8Ak,1,1493423880
1,mpd_pid_0,1AWQoqb9bSvzTjaLralEkT,1,1493423640
2,mpd_pid_0,7H6ev70Weq6DdpZyyTmUXk,1,1493423040
3,mpd_pid_0,2PpruBYCo4H7WOBJ7Q2EwM,1,1493416800
4,mpd_pid_0,4pmc2AxSEq6g7hPVlJCPyP,1,1493409480
...,...,...,...,...
11808549,mpd_pid_999996,2NrhLJdQJaga0YAI8Ipl16,0,1475599380
11808550,mpd_pid_999997,0pziAK2FOGW0h5m5gjMYAB,0,1498168380
11808551,mpd_pid_999997,46guedJC6IiQ8jsF3WkVLx,0,1498168380
11808552,mpd_pid_999997,4nKLRpFaGWQ6dwI5gaSys6,0,1498168380


In [18]:
data['user_click'] = data['exists']
data = data.sort_values(by=['user_id', 'timestamp']).reset_index(drop=True)
result = []
test_df = []
train_df = []
for user_id, user_data in data.groupby('user_id'):
    user_data = user_data.reset_index(drop=True)
    click_history = []
    seq_id = 0
    sep = int(0.9 * len(user_data)) // 10
    # Split into segments of 10 items
    for i in range(0, len(user_data), 10):
        if(i + 10 > len(user_data)):
            break
        segment = user_data[i:i+10]
        slate_of_items = segment['item_id'].tolist()
        user_clicks = segment['user_click'].tolist()

        row = {
          'user_id': user_id,
          'slate_of_items': slate_of_items,
          'user_mid': user_clicks,
          'user_mid_history': click_history[:],
          'sequence_id': seq_id
        }
        result.append(row)
        if seq_id < sep:
            train_df.append(row)
        else:
            test_df.append(row)
        # Update click history
        click_history.extend(segment[segment['user_click'] == 1]['item_id'].tolist())
        seq_id += 1

train_df = pd.DataFrame(train_df)
test_df = pd.DataFrame(test_df)
result_df = pd.DataFrame(result)
os.makedirs(path_to_output, exist_ok=True)
train_df.to_csv(os.path.join(path_to_output, 'train.csv') , sep='@', index=False)
test_df.to_csv(os.path.join(path_to_output, 'test.csv') , sep='@', index=False)

result_df.to_csv(os.path.join(path_to_output, 'all.csv') , sep='@', index=False)

## Extract item_info

In [9]:
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520053 entries, 0 to 520052
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           520053 non-null  object
 1   slate_of_items    520053 non-null  object
 2   user_mid          520053 non-null  object
 3   user_mid_history  520053 non-null  object
 4   sequence_id       520053 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 19.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306661 entries, 0 to 306660
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           306661 non-null  object
 1   slate_of_items    306661 non-null  object
 2   user_mid          306661 non-null  object
 3   user_mid_history  306661 non-null  object
 4   sequence_id       306661 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 11.7+ MB


In [13]:



# Build item_info.npy (N,D) and item_ids.npy (N,) from Spotify tracks CSV (NO dummy rows)
import os, numpy as np, pandas as pd

# expects: path_to_item (CSV), path_to_output (dir)
os.makedirs(path_to_output, exist_ok=True)

FEATURE_COLS = [
    "danceability","energy","key","loudness","mode","speechiness",
    "acousticness","instrumentalness","liveness","valence",
    "tempo","duration_ms","time_signature","year"
]

df = pd.read_csv(path_to_item)
missing = [c for c in ["id"] + FEATURE_COLS if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in tracks CSV: {missing}")

# optional min–max scale
feats = df[FEATURE_COLS].astype(float).copy()
for c in FEATURE_COLS:
    col = feats[c]
    cmin, cmax = float(col.min()), float(col.max())
    feats[c] = 0.0 if cmax == cmin else (col - cmin) / (cmax - cmin)
item_info = feats.astype(np.float32).values  # (N, D)

# save item_ids as fixed-width unicode (no pickle needed)
ids = df["id"].astype(str)
max_len = int(ids.str.len().max())
item_ids = ids.to_numpy(dtype=f"<U{max_len}")  # (N,)

np.save(os.path.join(path_to_output, "item_info.npy"), item_info)
np.save(os.path.join(path_to_output, "item_ids.npy"),  item_ids)

print("Saved:",
      os.path.join(path_to_output, "item_info.npy"), item_info.shape,
      "|", os.path.join(path_to_output, "item_ids.npy"), item_ids.shape)
print("Feature order:", FEATURE_COLS)



Saved: rl_agent/dataset/mdp/item_info.npy (1204025, 14) | rl_agent/dataset/mdp/item_ids.npy (1204025,)
Feature order: ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'year']


In [4]:
item_meta = np.load(os.path.join(path_to_output, "item_info.npy"))
item_meta.shape

(1204025, 14)

In [5]:
item_ids = np.load(os.path.join(path_to_output, "item_ids.npy"))
item_ids.shape

(1204025,)

In [17]:
# Filter interactions so every item_id exists in item_ids.npy (NO dummy, NO auto-fix)
import os, numpy as np, pandas as pd

# set your paths:
interactions_csv = os.path.join(path_to_data)  # change if different
filtered_interactions_csv = os.path.join(path_to_output, "interactions_filtered.csv")

item_ids = np.load(os.path.join(path_to_output, "item_ids.npy"))
allowed = set(item_ids.tolist())

inter = pd.read_csv(interactions_csv)
if "item_id" not in inter.columns:
    raise ValueError("interactions CSV must have an 'item_id' column")

inter["item_id"] = inter["item_id"].astype(str)
before = len(inter)
inter_f = inter[inter["item_id"].isin(allowed)].copy()
after = len(inter_f)

inter_f.to_csv(filtered_interactions_csv, index=False)
print(f"Kept {after}/{before} interactions ({after/before:.2%}).")
print("Saved ->", filtered_interactions_csv)


Kept 11808554/11808554 interactions (100.00%).
Saved -> rl_agent/dataset/mdp/interactions_filtered.csv


## Extract user info (just id)

In [5]:
data.head()

,user_id,item_id,exists,timestamp
0,mpd_pid_0,6I9VzXrHxO9rA9A5euc8Ak,1,1493423880
1,mpd_pid_0,1AWQoqb9bSvzTjaLralEkT,1,1493423640
2,mpd_pid_0,7H6ev70Weq6DdpZyyTmUXk,1,1493423040
3,mpd_pid_0,2PpruBYCo4H7WOBJ7Q2EwM,1,1493416800
4,mpd_pid_0,4pmc2AxSEq6g7hPVlJCPyP,1,1493409480


In [30]:
set_of_user = list(set(data['user_id'].to_list()))
set_of_user

['mpd_pid_235237',
 'mpd_pid_782836',
 'mpd_pid_360345',
 'mpd_pid_703905',
 'mpd_pid_431790',
 'mpd_pid_413031',
 'mpd_pid_397687',
 'mpd_pid_550120',
 'mpd_pid_920290',
 'mpd_pid_957627',
 'mpd_pid_72418',
 'mpd_pid_800971',
 'mpd_pid_40987',
 'mpd_pid_27491',
 'mpd_pid_749556',
 'mpd_pid_656313',
 'mpd_pid_548314',
 'mpd_pid_940169',
 'mpd_pid_358323',
 'mpd_pid_363218',
 'mpd_pid_692296',
 'mpd_pid_220983',
 'mpd_pid_780888',
 'mpd_pid_599077',
 'mpd_pid_187600',
 'mpd_pid_866263',
 'mpd_pid_112836',
 'mpd_pid_811842',
 'mpd_pid_749335',
 'mpd_pid_934298',
 'mpd_pid_428709',
 'mpd_pid_602505',
 'mpd_pid_609598',
 'mpd_pid_227597',
 'mpd_pid_292647',
 'mpd_pid_175667',
 'mpd_pid_536955',
 'mpd_pid_891572',
 'mpd_pid_954989',
 'mpd_pid_8949',
 'mpd_pid_170410',
 'mpd_pid_10185',
 'mpd_pid_951940',
 'mpd_pid_446340',
 'mpd_pid_875207',
 'mpd_pid_77940',
 'mpd_pid_40854',
 'mpd_pid_196234',
 'mpd_pid_51836',
 'mpd_pid_933021',
 'mpd_pid_725778',
 'mpd_pid_538437',
 'mpd_pid_839030',
 '

In [31]:
len(set_of_user)

887060

In [32]:
user_info = np.array(set_of_user).astype(str)
np.save(os.path.join(path_to_output, "user_info.npy"), user_info)
user_info.shape

(887060,)

In [34]:
user_meta = np.load(os.path.join(path_to_output, "user_info.npy"), allow_pickle=True)
user_meta

array(['mpd_pid_235237', 'mpd_pid_782836', 'mpd_pid_360345', ...,
       'mpd_pid_523913', 'mpd_pid_694897', 'mpd_pid_170852'],
      shape=(887060,), dtype='<U14')